In [2]:
import pandas as pd
from unidecode import unidecode
from difflib import get_close_matches

# --- Cargar datos (ejemplo: ajusta rutas si hace falta) ---
tabla1 = pd.read_csv("C:/Users/juana/Desktop/liquideuda_project/data_colegio/EPC1.csv")
tabla2 = pd.read_csv("C:/Users/juana/Downloads/ID GEO ESPAÑA - Hoja 1.csv")

# --- Preprocesar textos: quitar acentos y pasar todo a minúsculas ---
def limpiar(texto):
    return unidecode(str(texto)).lower().strip()

tabla1["Comunidad/Provincia_limpia"] = tabla1["Comunidad/Provincia"].apply(limpiar)
tabla2["Provincia_limpia"] = tabla2["Provincia"].apply(limpiar)
tabla2["CA_limpia"] = tabla2["Comunidad Autónoma"].apply(limpiar)

# --- Diccionarios auxiliares ---
dicc_provincias = dict(zip(tabla2["Provincia_limpia"], tabla2["ID GEO"]))
dicc_provincia_a_ca = dict(zip(tabla2["Provincia_limpia"], tabla2["CA_limpia"]))
dicc_ca = {v: k for k, v in zip(tabla2["CA_limpia"], tabla2["Comunidad Autónoma"])}  # limpio→original

# --- Función que busca coincidencia ---
def obtener_valor(valor):
    # 1) Intentamos buscar provincia
    posibles = get_close_matches(valor, dicc_provincias.keys(), n=1, cutoff=0.8)
    if posibles:
        return dicc_provincias[posibles[0]]  # Devuelve ID GEO

    # 2) Si no es provincia, probamos si es CA
    posibles_ca = get_close_matches(valor, dicc_ca.keys(), n=1, cutoff=0.8)
    if posibles_ca:
        return dicc_ca[posibles_ca[0]]  # Devuelve nombre de la CA (original con tildes)

    # 3) Si no coincide con nada, devolvemos "DESCONOCIDO"
    return "DESCONOCIDO"

# --- Crear nueva columna al inicio ---
tabla1.insert(0, "ID GEO / CA", tabla1["Comunidad/Provincia_limpia"].apply(obtener_valor))

# --- Limpiar columnas auxiliares ---
tabla1 = tabla1.drop(columns=["Comunidad/Provincia_limpia"])

# --- Resultado ---
print(tabla1.head())
df=tabla1


   ID GEO / CA  Unnamed: 0 Comunidad/Provincia Total  \
0  DESCONOCIDO           0           Andalucía   319   
1          104           1             Almería    22   
2          111           2               Cádiz    38   
3          114           3             Córdoba    27   
4          118           4             Granada    23   

  Personas Físicas SIN actividad empresarial  \
0                                      152.0   
1                                       10.0   
2                                       23.0   
3                                        9.0   
4                                       14.0   

  Personas Físicas CON actividad empresarial  Unnamed: 4 S. Limitadas  \
0                                       47.0         NaN          114   
1                                        4.0         NaN            8   
2                                        4.0         NaN           11   
3                                        6.0         NaN           10   
4        

In [3]:

# Función para limpiar nombres de columnas
def limpiar_columna(col):
    col = unidecode(col)             # quita acentos
    col = col.lower()                # pasa a minúsculas
    col = col.replace(" ", "_")      # espacios por _
    col = col.replace("/", "_")      # / por _
    col = col.replace("-", "_")      # - por _
    col = ''.join(c for c in col if c.isalnum() or c == "_")  # solo letras/números/_
    return col

# Aplicar a todas las columnas
df.columns = [limpiar_columna(c) for c in df.columns]

print(df.head())


   id_geo___ca  unnamed_0 comunidad_provincia total  \
0  DESCONOCIDO          0           Andalucía   319   
1          104          1             Almería    22   
2          111          2               Cádiz    38   
3          114          3             Córdoba    27   
4          118          4             Granada    23   

  personas_fisicas_sin_actividad_empresarial  \
0                                      152.0   
1                                       10.0   
2                                       23.0   
3                                        9.0   
4                                       14.0   

  personas_fisicas_con_actividad_empresarial  unnamed_4 s_limitadas  \
0                                       47.0        NaN         114   
1                                        4.0        NaN           8   
2                                        4.0        NaN          11   
3                                        6.0        NaN          10   
4                        

In [4]:
# Eliminar columnas que tengan todos los valores vacíos (NaN o "")
df = df.dropna(axis=1, how="all")           # elimina si todo es NaN
df = df.loc[:, (df != "").any(axis=0)]      # elimina si todo es cadena vacía

print("Columnas después de limpiar:", df.columns.tolist())
df


Columnas después de limpiar: ['id_geo___ca', 'unnamed_0', 'comunidad_provincia', 'total', 'personas_fisicas_sin_actividad_empresarial', 'personas_fisicas_con_actividad_empresarial', 's_limitadas', 's_anonimas', 'sotras', 'filename', 'personas_fisicas']


,id_geo___ca,unnamed_0,comunidad_provincia,total,personas_fisicas_sin_actividad_empresarial,personas_fisicas_con_actividad_empresarial,s_limitadas,s_anonimas,sotras,filename,personas_fisicas
0,DESCONOCIDO,0,Andalucía,319,152.0,47.0,114,5,1,2021_1T.xlsx,NaN
1,104,1,Almería,22,10.0,4.0,8,0,0,2021_1T.xlsx,NaN
2,111,2,Cádiz,38,23.0,4.0,11,0,0,2021_1T.xlsx,NaN
3,114,3,Córdoba,27,9.0,6.0,10,2,0,2021_1T.xlsx,NaN
4,118,4,Granada,23,14.0,3.0,6,0,0,2021_1T.xlsx,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1129,1620,1129,Gipuzkoa,93,54,14,23,1,1,2025_2T.xlsx,NaN
1130,DESCONOCIDO,1130,La Rioja,40,26,3,9,0,2,2025_2T.xlsx,NaN
1131,DESCONOCIDO,1131,Ceuta y Melilla,19,17,0,2,0,0,2025_2T.xlsx,NaN
1132,1851,1132,Ceuta,2,0,0,2,0,0,2025_2T.xlsx,NaN


In [5]:
# Eliminar la columna de índice si se importó como parte del DataFrame
df = df.reset_index(drop=True)

# Eliminar columna llamada 'unnamed_0' si existe
df = df.drop(columns=['unnamed_0'], errors='ignore')
df

,id_geo___ca,comunidad_provincia,total,personas_fisicas_sin_actividad_empresarial,personas_fisicas_con_actividad_empresarial,s_limitadas,s_anonimas,sotras,filename,personas_fisicas
0,DESCONOCIDO,Andalucía,319,152.0,47.0,114,5,1,2021_1T.xlsx,NaN
1,104,Almería,22,10.0,4.0,8,0,0,2021_1T.xlsx,NaN
2,111,Cádiz,38,23.0,4.0,11,0,0,2021_1T.xlsx,NaN
3,114,Córdoba,27,9.0,6.0,10,2,0,2021_1T.xlsx,NaN
4,118,Granada,23,14.0,3.0,6,0,0,2021_1T.xlsx,NaN
...,...,...,...,...,...,...,...,...,...,...
1129,1620,Gipuzkoa,93,54,14,23,1,1,2025_2T.xlsx,NaN
1130,DESCONOCIDO,La Rioja,40,26,3,9,0,2,2025_2T.xlsx,NaN
1131,DESCONOCIDO,Ceuta y Melilla,19,17,0,2,0,0,2025_2T.xlsx,NaN
1132,1851,Ceuta,2,0,0,2,0,0,2025_2T.xlsx,NaN


In [6]:
# Lista de columnas a convertir a numerico
cols_a_convertir = ["total", "s_limitadas", "s_anonimas", "sotras"]

# Convertir a numérico, los errores se convierten en NaN
for col in cols_a_convertir:
    df[col] = pd.to_numeric(df[col], errors="coerce")

print("Columnas convertidas a numéricas correctamente.")


Columnas convertidas a numéricas correctamente.


In [7]:
# 1. Convertimos a números (si hay texto raro lo pone como NaN)
df["personas_fisicas_sin_actividad_empresarial"] = pd.to_numeric(df["personas_fisicas_sin_actividad_empresarial"], errors="coerce")
df["personas_fisicas_con_actividad_empresarial"] = pd.to_numeric(df["personas_fisicas_con_actividad_empresarial"], errors="coerce")
df["personas_fisicas"] = pd.to_numeric(df["personas_fisicas"], errors="coerce")

# 2. Guardamos las filas que ya tienen datos en 'personas_fisicas' (para revisar después)
problemas = df[df["personas_fisicas"].notna()].copy()

# 3. Sumamos y rellenamos solo las filas vacías
df.loc[df["personas_fisicas"].isna(), "personas_fisicas"] = (
    df["personas_fisicas_sin_actividad_empresarial"].fillna(0) +
    df["personas_fisicas_con_actividad_empresarial"].fillna(0)
)
df


,id_geo___ca,comunidad_provincia,total,personas_fisicas_sin_actividad_empresarial,personas_fisicas_con_actividad_empresarial,s_limitadas,s_anonimas,sotras,filename,personas_fisicas
0,DESCONOCIDO,Andalucía,319.0,152.0,47.0,114.0,5.0,1.0,2021_1T.xlsx,199.0
1,104,Almería,22.0,10.0,4.0,8.0,0.0,0.0,2021_1T.xlsx,14.0
2,111,Cádiz,38.0,23.0,4.0,11.0,0.0,0.0,2021_1T.xlsx,27.0
3,114,Córdoba,27.0,9.0,6.0,10.0,2.0,0.0,2021_1T.xlsx,15.0
4,118,Granada,23.0,14.0,3.0,6.0,0.0,0.0,2021_1T.xlsx,17.0
...,...,...,...,...,...,...,...,...,...,...
1129,1620,Gipuzkoa,93.0,54.0,14.0,23.0,1.0,1.0,2025_2T.xlsx,68.0
1130,DESCONOCIDO,La Rioja,40.0,26.0,3.0,9.0,0.0,2.0,2025_2T.xlsx,29.0
1131,DESCONOCIDO,Ceuta y Melilla,19.0,17.0,0.0,2.0,0.0,0.0,2025_2T.xlsx,17.0
1132,1851,Ceuta,2.0,0.0,0.0,2.0,0.0,0.0,2025_2T.xlsx,0.0


In [8]:
# 1. Comprobar si hay campos vacíos en 'personas_fisicas'
if df["personas_fisicas"].isna().any():
    print("Atención: hay filas vacías en 'personas_fisicas'. Revisa antes de eliminar columnas.")
else:
    # 2. Si no hay vacíos, eliminar las columnas originales
    df = df.drop(columns=["personas_fisicas_sin_actividad_empresarial", "personas_fisicas_con_actividad_empresarial"])
    print("Columnas originales eliminadas, todo OK.")


Columnas originales eliminadas, todo OK.


In [9]:
df.dtypes

# Cantidad de vacíos y porcentaje por columna
vacios = df.isna().sum()  # cuenta NaN por columna
porcentaje = (vacios / len(df) * 100).round(2)  # calcula el % respecto al total de filas

# Crear un DataFrame resumido
resumen_vacios = pd.DataFrame({
    "Columnas": df.columns,
    "Cantidad vacíos": vacios.values,
    "Porcentaje vacíos": porcentaje.values
})

print(resumen_vacios)


              Columnas  Cantidad vacíos  Porcentaje vacíos
0          id_geo___ca                0               0.00
1  comunidad_provincia                0               0.00
2                total                4               0.35
3          s_limitadas                4               0.35
4           s_anonimas                4               0.35
5               sotras                4               0.35
6             filename                0               0.00
7     personas_fisicas                0               0.00


In [10]:
# Filtrar filas donde id_geo sea "DESCONOCIDO" (ignorando mayúsculas/minúsculas)
desconocidos = df[df['id_geo___ca'].str.upper() == 'DESCONOCIDO']

# Contar la cantidad de entradas por provincia dentro de desconocidos
resumen_provincias = desconocidos['comunidad_provincia'].value_counts().sort_values(ascending=False)

print(resumen_provincias)





comunidad_provincia
Andalucía                     18
Las Palmas de Gran Canaria    18
Cataluña                      18
Alicante                      18
Castellón                     18
A Coruña                      18
Pais Vasco                    18
La Rioja                      18
Ceuta y Melilla               18
Name: count, dtype: int64


In [11]:
# Encuentra el índice de las filas a eliminar
indices = df[df['id_geo___ca'] == 'DESCONOCIDO'].index

# Elimina esas filas
df.drop(indices, inplace=True)


In [12]:
# Cambiar nombre de la columna
df = df.rename(columns={"filename": "periodo"})

# Quitar todo lo que hay después del punto (.) en cada valor de la columna Periodo
df["periodo"] = df["periodo"].str.split(".").str[0]


In [13]:
df[["año", "trimestre"]] = df["periodo"].str.split("_", expand=True)
df["trimestre"] = df["trimestre"].str.extract(r'(\d+)')  # Extrae solo el número



In [14]:
df = df.drop(columns=["periodo"])

In [15]:
df['id_geo___ca'].value_counts()

id_geo___ca
639                     36
734                     36
943                     36
104                     18
816                     18
819                     18
845                     18
908                     18
917                     18
925                     18
comunitat valenciana    18
extremadura             18
1106                    18
1110                    18
galicia                 18
802                     18
1227                    18
1232                    18
1236                    18
1328                    18
1430                    18
1531                    18
1601                    18
1648                    18
1620                    18
1851                    18
813                     18
749                     18
castilla-la mancha      18
333                     18
114                     18
118                     18
121                     18
123                     18
129                     18
141                     18
222             

In [16]:
import os


# Filtrar solo las entradas NO numéricas
df_non_numeric = df[pd.to_numeric(df['id_geo___ca'], errors='coerce').isna()]

# Revisar valores únicos
print(df_non_numeric['id_geo___ca'].value_counts())

df = df[pd.to_numeric(df['id_geo___ca'], errors='coerce').notna()]
df_non_numeric = df[pd.to_numeric(df['id_geo___ca'], errors='coerce').isna()]
print(df_non_numeric['id_geo___ca'].value_counts())

id_geo___ca
castilla y leon         18
castilla-la mancha      18
comunitat valenciana    18
extremadura             18
galicia                 18
Name: count, dtype: int64
Series([], Name: count, dtype: int64)


In [17]:
# Definir ruta destino
ruta_destino = r"C:\Users\juana\Desktop\liquideuda_project\datasets_clean"

# Guardar CSV en la ruta destino
ruta_csv = os.path.join(ruta_destino, "EPC1_clean.csv")
df.to_csv(ruta_csv, index=False, encoding="utf-8-sig")

print(f"Archivo guardado en: {ruta_csv}")


Archivo guardado en: C:\Users\juana\Desktop\liquideuda_project\datasets_clean\EPC1_clean.csv


In [18]:
df.columns

Index(['id_geo___ca', 'comunidad_provincia', 'total', 's_limitadas',
       's_anonimas', 'sotras', 'personas_fisicas', 'año', 'trimestre'],
      dtype='object')